In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']="4,5"

In [2]:
%cd ../

/home/qwj/code/HippoRAG


PROMPTS

In [18]:
openie_post_ner_instruction = """Your task is to extract the relations from the given query and named entity lists. 
Respond with a JSON list of relations, with each relation representing a relationship helpful to solve the query. 
"""

openie_post_ner_frame = """
Query:
```
What language were books being translated into during the era of Haymo of Faversham?
```
{"named_entities":["books", "language", "Haymo of Faversham"]}
"""
ai_output = """
{"relations": ["being translated into", "during the era of"]}
"""

real_input_template = """
Query:
```
{passage}
```
{named_entity_json}
"""

def llama3_template(passage, named_entity_json):
    real_input = real_input_template.format(passage = passage, named_entity_json=named_entity_json)
    return "<|begin_of_text|><|start_header_id|>system<|end_header_id|>" + openie_post_ner_instruction + "<|eot_id|>" + \
    "<|start_header_id|>user<|end_header_id|>" + openie_post_ner_frame + "<|eot_id|>" + \
    "<|start_header_id|>assistant<|end_header_id|>" + ai_output + "<|eot_id|>" + \
    "<|start_header_id|>user<|end_header_id|>" + real_input + "<|eot_id|>" + \
     "<|start_header_id|>assistant<|end_header_id|>"

In [8]:
from vllm import LLM, SamplingParams
import pandas as pd

In [9]:
dataset = 'musique'
queries = pd.read_csv(f"output/{dataset}_queries.named_entity_output.tsv", sep='\t')
queries.head()

,Unnamed: 0,id,paragraphs,question,question_decomposition,answer,answer_aliases,answerable,triples
0,0,2hop__13548_13529,"[{'idx': 0, 'title': 'Lionel Messi', 'paragrap...",When was the person who Messi's goals in Copa ...,"[{'id': 13548, 'question': ""To whom was Messi'...",June 1982,[],True,"{""named_entities"": [""Messi"", ""Copa del Rey"", ""..."
1,1,3hop1__9285_5188_23307,"[{'idx': 0, 'title': 'Member states of NATO', ...",What month did the Tripartite discussions begi...,"[{'id': 9285, 'question': 'What was the nobili...",mid-June,[],True,"{""named_entities"": [""Britain"", ""France"", ""Wars..."
2,2,2hop__766973_770570,"[{'idx': 0, 'title': 'Minsk Region', 'paragrap...",What county is Erik Hort's birthplace a part of?,"[{'id': 766973, 'question': 'Erik Hort >> plac...",Rockland County,"['Rockland County, New York']",True,"{""named_entities"": [""Erik Hort""]}"
3,3,2hop__170823_120171,"[{'idx': 0, 'title': 'Blast Corps', 'paragraph...",What year did the publisher of Labyrinth end?,"[{'id': 170823, 'question': 'Labyrinth >> publ...",1986,[],True,"{""named_entities"": [""Labyrinth""]}"
4,4,2hop__511454_120259,"[{'idx': 0, 'title': 'Kavangoland', 'paragraph...",When was Lady Godiva's birthplace abolished?,"[{'id': 511454, 'question': 'Lady Godiva >> pl...",918,[],True,"{""named_entities"": [""Lady Godiva's birthplace""]}"


In [19]:
prompts = []
for i, row in queries.iterrows():
    passage = row.get('query', row.get('question'))
    named_entity_json = row['triples']
    prompts.append(llama3_template(passage, named_entity_json))
len(prompts)

1000

In [20]:
print(prompts[0])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>Your task is to extract the relations from the given query and named entity lists. 
Respond with a JSON list of relations, with each relation representing a relationship helpful to solve the query. 
<|eot_id|><|start_header_id|>user<|end_header_id|>
Query:
```
What language were books being translated into during the era of Haymo of Faversham?
```
{"named_entities":["books", "language", "Haymo of Faversham"]}
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
{"relations": ["being translated into", "during the era of"]}
<|eot_id|><|start_header_id|>user<|end_header_id|>
Query:
```
When was the person who Messi's goals in Copa del Rey compared to get signed by Barcelona?
```
{"named_entities": ["Messi", "Copa del Rey", "Barcelona"]}
<|eot_id|><|start_header_id|>assistant<|end_header_id|>


## Inference

In [12]:
model = LLM(model="/data/qwj/model/llama3_8b_instruct_hf", tensor_parallel_size=2, enable_prefix_caching=True)
sampling_params = SamplingParams(
    n=1, stop=['<|eot_id|>'], max_tokens=1000
)


2024-07-04 12:24:40,700	INFO worker.py:1749 -- Started a local Ray instance.


INFO 07-04 12:24:41 llm_engine.py:100] Initializing an LLM engine (v0.4.2) with config: model='/data/qwj/model/llama3_8b_instruct_hf', speculative_config=None, tokenizer='/data/qwj/model/llama3_8b_instruct_hf', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=/data/qwj/model/llama3_8b_instruct_hf)


You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 07-04 12:24:45 utils.py:660] Found nccl from library /home/qwj/.config/vllm/nccl/cu12/libnccl.so.2.18.1
(RayWorkerWrapper pid=444463) INFO 07-04 12:24:45 utils.py:660] Found nccl from library /home/qwj/.config/vllm/nccl/cu12/libnccl.so.2.18.1
INFO 07-04 12:24:47 selector.py:81] Cannot use FlashAttention-2 backend because the flash_attn package is not found. Please install it for better performance.
(RayWorkerWrapper pid=444463) INFO 07-04 12:24:46 selector.py:81] Cannot use FlashAttention-2 backend because the flash_attn package is not found. Please install it for better performance.
(RayWorkerWrapper pid=444463) INFO 07-04 12:24:46 selector.py:32] Using XFormers backend.
INFO 07-04 12:24:47 selector.py:32] Using XFormers backend.
INFO 07-04 12:25:03 pynccl_utils.py:43] vLLM is using nccl==2.18.1
(RayWorkerWrapper pid=444463) INFO 07-04 12:25:03 pynccl_utils.py:43] vLLM is using nccl==2.18.1
INFO 07-04 12:25:04 utils.py:132] reading GPU P2P access cache from /home/qwj/.config/vllm

(RayWorkerWrapper pid=444463) INFO 07-04 12:25:17 model_runner.py:1017] Graph capturing finished in 6 secs.


In [21]:
outputs = model.generate(prompts[:10], sampling_params)
output_text = [o.outputs[0].text for o in outputs]

Processed prompts: 100%|██████████| 10/10 [00:05<00:00,  1.85it/s]


In [22]:
output_text

['\n\n{"relations": ["compared to", "signed by", "goals of", "person who", "when"]}',
 '\n\n{"relations": ["Tripartite discussions begin", "between", "the country where", "despite being headquartered in the nation called", "originated"]}\nNote: Some of these relations may not be directly relevant to the query, but they are all extracted from the text. To solve the query effectively, more context or specific information would be required.',
 '\n\n{"relations": ["birthplace of", "part of a county"]}',
 '\n\n{"relations": ["publisher of", "end"]}',
 '\n\n{"relations": ["was abolished"]}',
 '\n\n{"relations": ["immediately north of", "region where", "location of the", "created"]}',
 '\n\n{"relations": ["is located next to the city where", "does the body of water break off into", "died in the city where", "is located by the city where"]}',
 '\n\nNote: There is no information about the explorer, the city, or the record label\'s headquarters in the named entities. We can\'t extract any specif

## Extract relations from results

In [23]:
import re
import json

def extract_json_dict(text):
    pattern = r'\{(?:[^{}]|(?:\{(?:[^{}]|(?:\{[^{}]*\})*)*\})*)*\}'
    match = re.search(pattern, text)

    if match:
        json_string = match.group()
        try:
            json_dict = json.loads(json_string)
            return json_dict
        except json.JSONDecodeError:
            return ''
    else:
        return ''

In [25]:
for i, t in enumerate(output_text):
    print(i, extract_json_dict(t))

0 {'relations': ['compared to', 'signed by', 'goals of', 'person who', 'when']}
1 {'relations': ['Tripartite discussions begin', 'between', 'the country where', 'despite being headquartered in the nation called', 'originated']}
2 {'relations': ['birthplace of', 'part of a county']}
3 {'relations': ['publisher of', 'end']}
4 {'relations': ['was abolished']}
5 {'relations': ['immediately north of', 'region where', 'location of the', 'created']}
6 {'relations': ['is located next to the city where', 'does the body of water break off into', 'died in the city where', 'is located by the city where']}
7 {'relations': ['reach', 'located', 'larger than']}
8 {'relations': ['the start of', 'battle of the', 'birthplace of', 'performer of III']}
9 


In [29]:
outputs = model.generate(prompts, sampling_params)
extracted_relations = [extract_json_dict(o.outputs[0].text) for o in outputs]

Processed prompts: 100%|██████████| 1000/1000 [00:23<00:00, 42.82it/s]


In [26]:
queries.columns


Index(['Unnamed: 0', 'id', 'paragraphs', 'question', 'question_decomposition',
       'answer', 'answer_aliases', 'answerable', 'triples'],
      dtype='object')

In [30]:
data_to_save = queries.copy()[['Unnamed: 0', 'id', 'question','triples']]
data_to_save['relations'] = extracted_relations
data_to_save.to_csv(f"output/{dataset}_queries.entity_and_relations.tsv", sep='\t')

In [31]:
data_to_save.head(10)

,Unnamed: 0,id,question,triples,relations
0,0,2hop__13548_13529,When was the person who Messi's goals in Copa ...,"{""named_entities"": [""Messi"", ""Copa del Rey"", ""...","{'relations': ['was signed by', 'comprised of'..."
1,1,3hop1__9285_5188_23307,What month did the Tripartite discussions begi...,"{""named_entities"": [""Britain"", ""France"", ""Wars...","{'relations': ['begin between', 'despite being..."
2,2,2hop__766973_770570,What county is Erik Hort's birthplace a part of?,"{""named_entities"": [""Erik Hort""]}","{'relations': ['birthplace of', 'part of a cou..."
3,3,2hop__170823_120171,What year did the publisher of Labyrinth end?,"{""named_entities"": [""Labyrinth""]}","{'relations': ['publisher of', 'end']}"
4,4,2hop__511454_120259,When was Lady Godiva's birthplace abolished?,"{""named_entities"": [""Lady Godiva's birthplace""]}",{'relations': ['was abolished']}
5,5,4hop2__71753_648517_70784_79935,When was the region immediately north of the r...,"{""named_entities"": [""Israel"", ""Battle of Qurah...","{'relations': ['region immediately north of', ..."
6,6,4hop1__94201_642284_131926_89261,Where does the body of water by the city where...,"{""named_entities"": [""Southeast Library"", ""Gulf...","{'relations': ['by the city of', 'into', 'wher..."
7,7,4hop1__152562_5274_458768_33633,When did the explorer reach the city where the...,"{""named_entities"": [""Vilaiyaadu Mankatha""]}","{'relations': ['headquarters is located', 'lar..."
8,8,3hop1__305282_282081_73772,When was the start of the battle of the birthp...,"{""named_entities"": [""battle of the birthplace""...","{'relations': ['start of the', 'of the birthpl..."
9,9,2hop__655505_110949,What is the Till dom ensamma performer's birth...,"{""named_entities"": [""Till dom ensamma""]}",
